In [49]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils, to_categorical

In [2]:
df = pd.read_csv("database_mscc.csv")

In [3]:
df.drop(columns = "Unnamed: 0", inplace=True)

In [4]:
for row in np.arange(0,df.shape[0]):
    df["y"][row] = df["y"][row][7:]

/tmp/ipykernel_17276/4108488883.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["y"][row] = df["y"][row][7:]


In [5]:
df.isna().sum()

y         0
0         0
1         0
2         0
3         0
       ... 
224    1439
225    1439
226    1439
227    1439
228    1440
Length: 230, dtype: int64

In [6]:
df = df.fillna(0)

In [11]:
df["y"].drop_duplicates()

0     surprised
1       fearful
2       disgust
14        happy
15         calm
18      neutral
34          sad
35        angry
Name: y, dtype: object

In [40]:
y = df["y"]
X = df.drop(columns="y")

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1405, stratify=y)

In [42]:
lb = LabelEncoder()
y_train_cat = np_utils.to_categorical(lb.fit_transform(y_train))
y_test_cat = np_utils.to_categorical(lb.fit_transform(y_test))

In [43]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [44]:
X_train_scaled_array = np.array(X_train_scaled)
X_test_scaled_array = np.array(X_test_scaled)
y_train_array = np.array(y_train_cat)
y_test_array = np.array(y_test_cat)

In [45]:
X_train_expanded_dims = np.expand_dims(X_train_scaled_array, axis=2)
X_test_expanded_dims = np.expand_dims(X_test_scaled_array, axis=2)

In [85]:
model = models.Sequential()
model.add(layers.Conv1D(256, 8, padding="same", input_shape=(X_train_expanded_dims.shape[1],1), activation = "tanh"))
model.add(layers.Conv1D(256, 8, padding='same', activation = "relu"))
# model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
# model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(64, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(64, 8, padding='same', activation = "relu"))
model.add(layers.Flatten())
model.add(layers.Dense(8, activation = "softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [82]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_64 (Conv1D)          (None, 230, 256)          2304      
                                                                 
 conv1d_65 (Conv1D)          (None, 230, 256)          524544    
                                                                 
 dropout_16 (Dropout)        (None, 230, 256)          0         
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 28, 256)          0         
 g1D)                                                            
                                                                 
 conv1d_66 (Conv1D)          (None, 28, 128)           262272    
                                                                 
 conv1d_67 (Conv1D)          (None, 28, 128)           131200    
                                                      

In [86]:
es = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(X_train_expanded_dims, y_train_array, validation_split=0.3, epochs=500, batch_size=32, callbacks=[es])

Epoch 1/500
23/23 [==============================] - 14s 319ms/step - loss: 2.0695 - accuracy: 0.1348 - val_loss: 2.0735 - val_accuracy: 0.1089
Epoch 2/500
23/23 [==============================] - 6s 251ms/step - loss: 2.0557 - accuracy: 0.1589 - val_loss: 2.0391 - val_accuracy: 0.1617
Epoch 3/500
23/23 [==============================] - 6s 254ms/step - loss: 2.0048 - accuracy: 0.1986 - val_loss: 1.9707 - val_accuracy: 0.2508
Epoch 4/500
23/23 [==============================] - 6s 242ms/step - loss: 1.9553 - accuracy: 0.2170 - val_loss: 1.8925 - val_accuracy: 0.2475
Epoch 5/500
23/23 [==============================] - 5s 239ms/step - loss: 1.8767 - accuracy: 0.2752 - val_loss: 1.8560 - val_accuracy: 0.2442
Epoch 6/500
23/23 [==============================] - 7s 310ms/step - loss: 1.8480 - accuracy: 0.2610 - val_loss: 1.8403 - val_accuracy: 0.2805
Epoch 7/500
23/23 [==============================] - 7s 292ms/step - loss: 1.8797 - accuracy: 0.2752 - val_loss: 1.8854 - val_accuracy: 0.244

Epoch 58/500
23/23 [==============================] - 6s 262ms/step - loss: 0.2472 - accuracy: 0.9035 - val_loss: 0.1932 - val_accuracy: 0.9538
Epoch 59/500
23/23 [==============================] - 6s 257ms/step - loss: 0.1723 - accuracy: 0.9404 - val_loss: 0.1706 - val_accuracy: 0.9637
Epoch 60/500
23/23 [==============================] - 6s 254ms/step - loss: 0.2013 - accuracy: 0.9191 - val_loss: 0.7225 - val_accuracy: 0.6634
Epoch 61/500
23/23 [==============================] - 6s 250ms/step - loss: 0.4686 - accuracy: 0.8227 - val_loss: 0.3342 - val_accuracy: 0.9307
Epoch 62/500
23/23 [==============================] - 6s 253ms/step - loss: 0.1643 - accuracy: 0.9489 - val_loss: 0.2938 - val_accuracy: 0.9076
Epoch 63/500
23/23 [==============================] - 6s 257ms/step - loss: 0.3419 - accuracy: 0.8667 - val_loss: 0.7847 - val_accuracy: 0.6304
Epoch 64/500
23/23 [==============================] - 6s 261ms/step - loss: 0.8884 - accuracy: 0.7121 - val_loss: 0.6056 - val_accuracy:

In [87]:
model.evaluate(X_test_expanded_dims, y_test_array)

14/14 [==============================] - 1s 55ms/step - loss: 0.2386 - accuracy: 0.9630


[0.23858296871185303, 0.9629629850387573]